In [ ]:
import numpy as np
import AuxFunctions as aux
import main as main
import timeit
import sys
sys.path.append("/home/s2250115/petsc/arch-linux-c-debug/lib")

In [ ]:
# Define the parameters of the system
N = 1000 #Number of seeds
box = [-3, -3, 0, 3, 3, 1] #list or tuple defining domain [xmin, ymin, zmin, xmax, ymax, zmax]
per_tol = 1e-3 #Percent tolerance
tf = 1 #Final time
Ndt = 100 #Number of timesteps

#Initialize initial condition
Z = aux.create_initial(N, 4, 4, 4, -4, -4, 0, 'lattice wsp') #Initial guess that is uniform in x any y and sin(uniform) in z

In [ ]:
#start = timeit.default_timer()
main.SGSolver(box, Z, N, per_tol, tf, Ndt) #Solve and save the solution
#end = timeit.default_timer()
#print(abs(end - start))

In [ ]:
aux.animator('./SG_data.npz', 'C', '2D', tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
aux.animator('./SG_data.npz', 'C', '3D', tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
aux.animator('./SG_data.npz', 'Z', '2D', tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
aux.animator('./SG_data.npz', 'Z', '3D', tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_size_inches(10, 10, True)
#ax = fig.add_subplot()
ax = fig.add_subplot(projection='3d')
ax.scatter(Z[:,0], Z[:,1], Z[:,2], c = Z[:,2], cmap = 'jet', edgecolor = 'none', s = 8)
ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])
ax.set_zlim([-10, 10])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=0, azim=0)

plt.show()

In [ ]:
from pysdot.domain_types import ConvexPolyhedraAssembly
from pysdot import PowerDiagram
import pyvista as pv
import imageio.v2 as iio

box = [-3, -3, 0, 3, 3, 1]
domain = ConvexPolyhedraAssembly()
domain.add_box([box[0], box[1], box[2]], [box[3], box[4], box[5]])

# Load the data
loaded_data = np.load('SG_data.npz')

# Access the individual arrays
Z = loaded_data['data1']
C = loaded_data['data2']
w = loaded_data['data3']

#Compute Meridonal Velocities?
f = 1e-4
M = [[0] * len(w[0]) for _ in range(len(w))]
for i in range(len(w)):
    for j in range(len(w[0])):
        M[i][j] = f * (Z[i][j][0] - C[i][j][0])

# Set up the animation parameters
n_frames = len(w)  # Number of frames

# Create an empty list to store frames
frames = []

# Generate frames for the animation
for i in range(n_frames):

    #Draw the tessellation
    pd = PowerDiagram(positions = Z[i] , weights = w[i] , domain = domain)

    # Store the volumes in an array
    vols = np.array(M[i])

    # Save the results in a .vtk file
    filename = "results.vtk"
    pd.display_vtk(filename)

    # Read the data
    grid=pv.read(filename)

    # create cell data that gives the cell volumes, this allows us to colour by cell volumes
    cell_vols = vols[grid.cell_data['num'].astype(int)]
    grid.cell_data['vols']=cell_vols

    # plot the data with an automatically created plotter, for a static picture use backend='static'
    plotter = pv.Plotter(window_size=[800,800], notebook = False, off_screen=True)
    plotter.add_mesh(grid)

    # Render the frame
    plotter.show()

    # Add a headlight
    light=pv.Light(light_type='headlight')
    plotter.add_light(light)

    # Get the frame as an image array
    frame = plotter.screenshot(transparent_background=True)

    # Add the frame to the list of frames
    frames.append(frame)

# Save the frames as an animation file
output_file = 'animation.gif'
iio.mimwrite(output_file, frames, format='gif', duration = 40)

In [ ]:
from pysdot import OptimalTransport
from pysdot import PowerDiagram

Num = [8, 27, 64, 125, 216, 343, 512, 729, 1000, 1331, 1728, 2197, 2744, 3375]
box = [-3, -3, -3, 3, 3, 3]
D = aux.make_domain(box)

for i in Num:
    Z = aux.create_initial(i, 1, 1, 1, 0, 0, 0, 'lattice') #Initial guess that is uniform in x any y and sin(uniform) in z
    w0 = aux.Rescale_weights(box, Z)[0]
    ot = OptimalTransport(positions = Z, weights = w0, masses = D.measure() / i * np.ones(i), domain = D, linear_solver= 'Scipy')
    start = timeit.default_timer()
    #pd = PowerDiagram(positions=Z,weights=w0,domain=D)
    ot.adjust_weights()
    end = timeit.default_timer()
    print(i, abs(end - start))
